# Point Source Tutorial

This is a quick demonstration of how to model both extended objects and point sources in the same scence. After more testing we hope to replace this with a more robust demonstration of crowded field photometry, using an iterative detection/deblending procedure. In the meantime feel free to create your own algorithm for crowded fields and let us know how it goes on the [DESC Blending](https://lsstc.slack.com/messages/desc-blending) channel on slack.

First we load a simulated image where we know the true value of all of the objects. This allows us to know which sources are galaxies and which ones are stars so we can use the appropriate source type.

In [ ]:
# Import Packages and setup
import logging
import os

import numpy as np

import scarlet
import scarlet.display
from scarlet.constraint import Normalization

# For display purposes
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')

In [ ]:
# Use this to point to the location of the data on your system
datapath = "../../data/unmatched_sim"
psfs = np.load(os.path.join(datapath, "psfs.npz"))["psfs"]
data = np.load(os.path.join(datapath, "images.npz"))
images = data["images"]
filters = data["filters"]

# Load the catalog
from astropy.table import Table as ApTable
catalog = ApTable.read(os.path.join(datapath, "true_catalog.fits"))
# Only use a single component for each source
catalog = catalog[catalog["component"]!="disk"]
bg_rms = np.array([20]*len(images))

# display psfs
pnorm = scarlet.display.Asinh(img=psfs, Q=20)
filter_indices = [3,2,1]
prgb = scarlet.display.img_to_rgb(psfs, filter_indices=filter_indices, norm=pnorm)
plt.imshow(prgb)
plt.show()

# Use Asinh scaling for the images
norm = scarlet.display.Asinh(img=images, Q=20)
# Map i,r,g -> RGB
# Convert the image to an RGB image
img_rgb = scarlet.display.img_to_rgb(images, filter_indices=filter_indices, norm=norm)
plt.imshow(img_rgb)
for src in catalog:
    if src["is_star"]:
        plt.plot(src["x"], src["y"], "rx", mew=2)
    else:
        plt.plot(src["x"], src["y"], "bx", mew=2)
plt.show()

## Fit the PSFs

To avoid artifacts that arise when performing a full deconvolution on exteneded objects, we choose a small "target" PSF and calculate the difference kernel to match the PSF in each band to the target. See [Matching PSF's](psf_matching.ipynb) for a more detailed explanation of PSF matching.

In [ ]:
# Get the target PSF to partially deconvolve the image psfs
target_psf = scarlet.psf_match.fit_target_psf(psfs, scarlet.psf_match.moffat)
# Display the target PSF
plt.imshow(target_psf,norm=scarlet.display.Asinh(img=target_psf))
plt.show()
# Match each PSF to the target PSF
diff_kernels, psf_blend = scarlet.psf_match.build_diff_kernels(psfs, target_psf)

## Initialize the sources

Here is where we define the sources. In this case we know which ones are stars and which are galaxies, which may or may not be realistic depending on the depth and locations of the images taken. For example, at depths shallower than GAIA it should be possible to flag most of the stars in regions outside the galactic bulge, while in more crowded fields or long exposures a different method might be needed (such as color priors on stars vs galaxies) to determine which sources to model as point sources and which ones to model as extended objects.

In [ ]:
import scarlet.constraint as sc

# We turn off acceleration, since it can lead to erratic behavior with certain constraints
# More testing will hopefully fix these issues.
config = scarlet.config.Config(refine_skip=10, accelerated=False)

# This is the constraint to use for a point source, which doesn't
# require the more complicated constraints used for extended sources
constraints = (sc.SimpleConstraint(normalization=sc.Normalization.S))

# Initalize the sources
sources = []
for src in catalog:
    if src["is_star"]:
        new_source = scarlet.PointSource(
            (src["y"], src["x"]),
            images,
            fix_morph=True, # This is required to model a point source
            constraints = constraints,
            psf=psfs, # full PSF convolution from a single point
            shape=psfs[0].shape,
            normalization=Normalization.S # Use S normalization
            #shift_center=0
        )
    else:
        new_source = scarlet.ExtendedSource(
            (src["y"], src["x"]),
            images,
            bg_rms,
            psf=diff_kernels, # convolve from target PSF
            normalization=sc.Normalization.S, # for consistency with the point sources
            #shift_center=0
        )
    sources.append(new_source)

Notice that we fix the morphology (`fix_morph=True`) and normalize the $S$ matrix for point sources. This allows us to only set the central pixel and never update it. This is described in more detail in the [User Guide](../user_docs.ipynb#Normalization). You'll also notice that we passed the point sources the full PSF in each band, whereas for the extended sources we pass the difference kernel. This is because a point source is a fully deconvolved representation of the object while extended sources are partially deconvolved. It also means that we have to be careful if we have a scene like this that is a mixture of extended sources and point sources. In order to have a consistent sparse (partially deconvolved) representation of the scene we need to convolve the point sources with the _target PSF_, not the full PSF that matches the image, once deblending has been completed to generate our model.

Also notice that we used the same $S$ normalization for both the point sources and extended sources, which allows us to have a consistent definition of $A$ and $S$ for all sources, regardless of their type.

## Create the blend and initialize the sources

In [ ]:
# Initialize the Blend object, which later fits the model
blend = scarlet.Blend(sources)
blend.set_data(images, bg_rms=bg_rms, config=config)

# Display the initial model
model = blend.get_model()
img_rgb = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=norm)
plt.imshow(img_rgb)
for src in catalog:
    if src["is_star"]:
        plt.plot(src["x"], src["y"], "rx", mew=2)
    else:
        plt.plot(src["x"], src["y"], "bx", mew=2)
plt.show()

Our three stars (the red x's) are initialized to match their peak value with the peak of the image while the extended sources are initialized in the usual way.

## Fit the model and display the results

In [ ]:
%%time
blend.fit(200, e_rel=1e-3)
print("Fit for {0} iterations".format(blend.it))

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = [fig.add_subplot(1,3,n+1) for n in range(3)]

# Display the data
img_rgb = scarlet.display.img_to_rgb(images, filter_indices=filter_indices, norm=norm)
ax[0].imshow(img_rgb)
ax[0].set_title("Data")
for src in catalog:
    if src["is_star"]:
        ax[0].plot(src["x"], src["y"], "rx", mew=2)
    else:
        ax[0].plot(src["x"], src["y"], "bx", mew=2)

#Display the model
model = blend.get_model()
img_rgb = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=norm)
ax[1].imshow(img_rgb)
ax[1].set_title("Model")
for src in blend.sources:
    y, x = src.components[0].center
    ax[1].plot(x, y, "gx", mew=2)

# Display the residual
residual = images-model
img_rgb = scarlet.display.img_to_rgb(residual, filter_indices=filter_indices)
ax[2].imshow(img_rgb)
ax[2].set_title("Residual")

# Show the plots
plt.show()

# Show the morphologies
for src in blend.sources:
    plt.imshow(src.components[0].morph)
    plt.show()

print("Total residual {0}".format(np.sum(np.abs(residual))))

## Comparison with Extended Sources

We now use the same procedure to model all of the sources as extended sources.

In [ ]:
sources = [
    scarlet.source.ExtendedSource(
        (src['y'],src['x']),
        images,
        bg_rms,
        psf=diff_kernels,
        normalization=sc.Normalization.S
    ) for src in catalog
]

blend = scarlet.Blend(sources)
blend.set_data(images, bg_rms=bg_rms, config=config)

In [ ]:
%%time
blend.fit(200, e_rel=1e-3)
print("Fit for {0} iterations".format(blend.it))

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = [fig.add_subplot(1,3,n+1) for n in range(3)]

# Display the data
img_rgb = scarlet.display.img_to_rgb(images, filter_indices=filter_indices, norm=norm)
ax[0].imshow(img_rgb)
ax[0].set_title("Data")
for src in catalog:
    if src["is_star"]:
        ax[0].plot(src["x"], src["y"], "rx", mew=2)
    else:
        ax[0].plot(src["x"], src["y"], "bx", mew=2)

#Display the model
model = blend.get_model()
img_rgb = scarlet.display.img_to_rgb(model, filter_indices=[3,2,1], norm=norm)
ax[1].imshow(img_rgb)
ax[1].set_title("Model")
for src in blend.sources:
    y, x = src.components[0].center
    ax[1].plot(x, y, "gx", mew=2)

# Display the residual
residual = images-model
img_rgb = scarlet.display.img_to_rgb(residual, filter_indices=filter_indices)
ax[2].imshow(img_rgb)
ax[2].set_title("Residual")

# Show the plots
plt.show()

print("Total residual {0}".format(np.sum(np.abs(residual))))

In [ ]:
for src in blend.sources:
    plt.imshow(src.components[0].morph)
    plt.show()

So we see improvement in computation time, total flux (residual), and galaxy shapes using point sources when we know they can be used.